# 채팅 완료 클라이언트 사용 방법

EasyLLM은 `gpt-3.5-turbo` 및 `gpt-4`를 오픈 소스 모델로 대체하는 추상화 계층으로 사용할 수 있습니다.

클라이언트만 변경하여 OpenAI API에서 자체 애플리케이션을 변경할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 사용하고 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 예제 API 호출을 통해 채팅 형식을 보여줍니다.

### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm 

In [6]:
# EasyLLM API를 호출하기 위한 EasyLLM Python 라이브러리를 가져옵니다.
import easyllm

### 2. 채팅 API 호출 예제

채팅 API 호출에는 두 가지 필수 입력이 있습니다.
- `model`: 사용하려는 모델의 이름(예: `meta-llama/Llama-2-70b-chat-hf`) 또는 API만 호출하려면 비워 둡니다.
- `messages`: 메시지 개체 목록이며 각 개체에는 두 가지 필수 필드가 있습니다.
    - `role`: 메신저의 역할( `system`, `user` 또는 `assistant`)
    - `content`: 메시지 내용(예: `아름다운 시를 써주세요`)

OpenAI API와 비교하여 `huggingface` 모듈은 프롬프트 및 중지 시퀀스를 사용자 지정하는 데 사용할 수 있는 `prompt_builder` 및 `stop_sequences` 매개변수도 노출합니다. EasyLLM 패키지에는 프롬프트 빌더 유틸리티가 함께 제공됩니다.

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [1]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["HUGGINGFACE_PROMPT"] = "llama2" # vicuna, wizardlm, stablebeluga, open_assistant
# os.environ["HUGGINGFACE_TOKEN"] = "hf_xxx" 

from easyllm.clients import huggingface

# 환경 변수를 사용하지 않고 구성 변경
# huggingface.api_key="hf_xxx"
# huggingface.prompt_builder = "llama2"


MODEL="meta-llama/Llama-2-70b-chat-hf"

response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "\n당신은 도움이 되고 정중하며 정직한 조수입니다. 항상 안전하면서 가능한 한 도움이 되도록 답변하십시오. 답변에는 유해하거나 비윤리적이거나 인종 차별적이거나 성 차별적이거나 유독하거나 위험하거나 불법적인 내용이 포함되어서는 안 됩니다. 답변이 사회적으로 편향되지 않고 긍정적인지 확인하십시오.\n\n질문이 이해가 되지 않거나 사실적으로 일관성이 없는 경우 올바르지 않은 내용을 답변하는 대신 이유를 설명하십시오. 질문에 대한 답변을 모르는 경우 거짓 정보를 공유하지 마십시오."},
        {"role": "user", "content": "똑똑."},
        {"role": "assistant", "content": "누구세요?"},
        {"role": "user", "content": "고양이."},
    ],
      temperature=0.9,
      top_p=0.6,
      max_tokens=1024,
)
response

{'id': 'hf-lt8HWKZn-O',
 'object': 'chat.completion',
 'created': 1695106434,
 'model': 'meta-llama/Llama-2-70b-chat-hf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': ' 고양이 누구?'},
   'finish_reason': 'eos_token'}],
 'usage': {'prompt_tokens': 149, 'completion_tokens': 5, 'total_tokens': 154}}

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `chat.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `message`: 모델에서 생성한 메시지 개체( `role` 및 `content` 포함)
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `index`: 선택 항목 목록에서 완성의 인덱스

다음을 사용하여 회신만 추출합니다.

In [3]:
print(response['choices'][0]['message']['content'])

 고양이 누구?


대화 기반이 아닌 작업도 첫 번째 사용자 메시지에 지침을 배치하여 채팅 형식에 맞출 수 있습니다.

예를 들어 모델에게 해적 블랙비어드 스타일로 비동기 프로그래밍을 설명하도록 요청하려면 다음과 같이 대화를 구성할 수 있습니다.

In [4]:
# 시스템 메시지가 있는 예제
response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "당신은 도움이 되는 조수입니다."},
        {"role": "user", "content": "수학 교사 스타일로 비동기 프로그래밍을 설명해주세요."},
    ],
)

print(response['choices'][0]['message']['content'])


 안녕하세요, 학생 여러분! 오늘 우리는 프로그램의 효율성과 응답성을 높이는 데 도움이 되는 흥미로운 주제인 비동기 프로그래밍에 대해 배울 것입니다.

여러 사람과 함께 프로젝트를 진행하고 있고 다른 사람이 시작하기 전에 자신의 부분을 끝내야 한다고 상상해 보십시오. 하지만 자신의 부분을 시작하기 위해 다른 사람이 자신의 부분을 끝내기를 기다리고 있습니다. 이것은 비동기 프로그래밍이 작동하는 방식과 유사합니다.

비동기 프로그래밍에서는 프로그램을 "작업"이라는 더 작은 부분으로 나눕니다. 이러한 작업은 다른 작업이 실행되는 것을 차단하지 않고 독립적으로 실행될 수 있습니다. 즉, 한 작업이 서버의 응답이나 사용자 입력과 같이 무언가가 발생하기를 기다리는 경우에도 다른 작업은 그동안 계속 실행될 수 있습니다.

이를 설명하기 위해 간단한 예를 사용해 보겠습니다. 샌드위치를 만들고 있다고 상상해 보십시오. 빵 조각을 함께 놓고 속을 채운 다음 샌드위치를 냉장고에 넣어 식혀야 합니다. 하지만 빵이 구워질 때까지 샌드위치를 만들기 시작할 수 없고 조립될 때까지 샌드위치를 냉장고에 넣을 수 없습니다.

이 시나리오에서 빵을 굽고 샌드위치를 조립하는 것은 두 가지 별개의 작업입니다. 동기적으로 수행한다면 다음과 같이 차례로 수행할 것입니다.

1. 빵 굽기
2. 샌드위치 조립하기
3. 샌드위치를 냉장고에 넣기

하지만 비동기 프로그래밍을 사용하면 다음과 같이 동시에 수행할 수 있습니다.

1. 빵 굽기 (시작)
2. 샌드위치 조립하기 (시작)
3. 빵 굽기 (완료)
4. 샌드위치를 냉장고에 넣기

작업을 동시에 수행하면 시간을 절약하고 프로그램의 효율성을 높일 수 있습니다. 하지만 일이 일어나는 순서에 대해 혼동하지 않도록 주의해야 합니다. 그래서 모든 것을 추적하기 위해 "프로미스"와 "콜백"과 같은 특수 도구를 사용합니다.

그러니 학생 여러분, 이것이 비동기 프로그래밍을 조금 더 잘 이해하는 데 도움이 되기를 바랍니다. 기억하십시오. 모든 것은 프로그램을 더 작고 

In [5]:
# 시스템 메시지가 없고 디버그 플래그가 켜진 예제:
response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "해적 블랙비어드 스타일로 비동기 프로그래밍을 설명해주세요."},
    ],
    debug=True,
)

print(response['choices'][0]['message']['content'])


08/04/2023 08:16:57 - DEBUG - easyllm.utils - 모델로 전송될 프롬프트:
<s>[INST] 해적 블랙비어드 스타일로 비동기 프로그래밍을 설명해주세요. [/INST]
08/04/2023 08:16:57 - DEBUG - easyllm.utils - URL:
https://api-inference.huggingface.co/models/meta-llama/Llama-2-70b-chat-hf
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 중지 시퀀스:
[]
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 생성 매개변수:
{'do_sample': True, 'return_full_text': False, 'max_new_tokens': 1024, 'top_p': 0.6, 'temperature': 0.9, 'stop_sequences': [], 'repetition_penalty': 1.0, 'top_k': 10, 'seed': 42}
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 인덱스 0의 응답:
index=0 message=ChatMessage(role='assistant', content=' 아호이, 친구! 비동기 프로그래밍 이야기를 찾고 있나? 그럼 그로그 한 잔 들고 앉아서 잘 들어보게, 블랙비어드가 해줄 이야기가 있으니.\n\n비동기 프로그래밍이란, 친구들, 위험한 바다를 항해하는 것과 같네. 정신을 바짝 차리고 수면 아래 숨겨진 위험을 조심해야 하네.\n\n항해 중인데 갑자기 거대한 폭풍이 몰아친다고 상상해보게. 바람이 울부짖고 파도가 부딪히고 배에 물이 차오르네. 이제 빨리 행동하지 않으면 데이비 존스의 사물함으로 보내질 걸세!\n\n하지만, 친구들, 배를 버릴 수는 없네. 해치를 닫고 폭풍을 이겨내야 하네. 그리고 바로 여기서 비동기 프로그래밍이 등장하네.\n\n비동기 프로그래밍은 돛을 올리고 바람이 일을 하도록

### 3. 소수 샷 프롬프트

경우에 따라 원하는 것을 모델에게 말하는 것보다 보여주는 것이 더 쉽습니다.

원하는 것을 모델에게 보여주는 한 가지 방법은 가짜 예제 메시지를 사용하는 것입니다.

예를 들어:

In [6]:
# 비즈니스 전문 용어를 더 간단한 말로 번역하도록 모델을 준비시키는 가짜 소수 샷 대화 예제
response = huggingface.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "당신은 도움이 되고 패턴을 따르는 조수입니다."},
        {"role": "user", "content": "다음 기업 전문 용어를 평이한 영어로 번역하는 것을 도와주세요."},
        {"role": "assistant", "content": "물론입니다, 기꺼이 도와드리겠습니다!"},
        {"role": "user", "content": "새로운 시너지가 최고 수익 성장을 이끌 것입니다."},
        {"role": "assistant", "content": "함께 잘 작동하는 것들이 수익을 증가시킬 것입니다."},
        {"role": "user", "content": "증가된 레버리지 기회에 대해 논의할 시간이 더 있을 때 다시 논의합시다."},
        {"role": "assistant", "content": "더 잘하는 방법에 대해 덜 바쁠 때 나중에 이야기합시다."},
        {"role": "user", "content": "이 늦은 전환은 고객 결과물에 대해 바다를 끓일 시간이 없다는 것을 의미합니다."},
    ],
)

print(response["choices"][0]["message"]["content"])


08/04/2023 08:16:57 - DEBUG - easyllm.utils - 모델로 전송될 프롬프트:
<s>[INST] <<SYS>>
당신은 도움이 되고 패턴을 따르는 조수입니다.
<</SYS>>

다음 기업 전문 용어를 평이한 영어로 번역하는 것을 도와주세요. [/INST] 물론입니다, 기꺼이 도와드리겠습니다!</s><s>[INST] 새로운 시너지가 최고 수익 성장을 이끌 것입니다. [/INST] 함께 잘 작동하는 것들이 수익을 증가시킬 것입니다.</s><s>[INST] 증가된 레버리지 기회에 대해 논의할 시간이 더 있을 때 다시 논의합시다. [/INST] 더 잘하는 방법에 대해 덜 바쁠 때 나중에 이야기합시다.</s><s>[INST] 이 늦은 전환은 고객 결과물에 대해 바다를 끓일 시간이 없다는 것을 의미합니다. [/INST]
08/04/2023 08:16:57 - DEBUG - easyllm.utils - URL:
https://api-inference.huggingface.co/models/meta-llama/Llama-2-70b-chat-hf
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 중지 시퀀스:
[]
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 생성 매개변수:
{'do_sample': True, 'return_full_text': False, 'max_new_tokens': 1024, 'top_p': 0.6, 'temperature': 0.9, 'stop_sequences': [], 'repetition_penalty': 1.0, 'top_k': 10, 'seed': 42}
08/04/2023 08:16:57 - DEBUG - easyllm.utils - 인덱스 0의 응답:
index=0 message=ChatMessage(role='assistant', content=" 너무 늦게 방향을 바꿔서 고객을 위해 완벽한 작업을 할 시간이 없습니다.") finish_rea

대화 엔지니어링에 대한 모든 시도가 처음부터 성공하는 것은 아닙니다.

첫 번째 시도가 실패하면 모델을 준비하거나 조정하는 다양한 방법을 실험하는 것을 두려워하지 마십시오.

예를 들어, 한 개발자는 모델이 더 높은 품질의 응답을 제공하도록 조정하는 데 도움이 되도록 "지금까지 훌륭했습니다. 완벽했습니다."라는 사용자 메시지를 삽입했을 때 정확도가 증가하는 것을 발견했습니다.

모델의 신뢰성을 높이는 방법에 대한 더 많은 아이디어는 [신뢰성 향상 기술](../techniques_to_improve_reliability.md)에 대한 가이드를 참조하십시오. 비채팅 모델용으로 작성되었지만 많은 원칙이 여전히 적용됩니다.